In [1]:
# !pip install jsonlines
# !pip install PySimpleGUI

In [2]:
import numpy as np
import pandas as pd
import json
import jsonlines
import typer
import PySimpleGUI as sg

from scipy.linalg import svd
from sklearn.model_selection import train_test_split

import os
import time
import re
from PIL import Image
import io
import requests
import json
import jsonlines

import selenium
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Config

In [3]:
crawled_path = 'CrawledPapers.json'

# 1

In [4]:
def crawl():
    driver = webdriver.Firefox()

    infos = dict()
    ids = set()

    # try:
    #     with jsonlines.open('CrawledPapers.json') as reader:
    #         for obj in reader:
    #             ids.add(obj['id'])
    # except:
    #     pass

    List = []

    i = 0
    json_length = 0
    while json_length < 2000:
        try:
            info = dict()

            file1 = open('start.txt', 'r')
            Lines = file1.readlines()
            file1.close()
            url = Lines[i]

            # extract and check id
            id = re.search(r'^(https://)?(academic\.microsoft\.com/)?(paper/)?(\d+)', url).group(4)
            if id in ids:
                i += 1
                continue
            info['id'] = id
            ids.add(id)

            driver.get(url)

            time.sleep(4)

            # extract name
            name = driver.find_element_by_css_selector("h1.name").text
            info['name'] = name

            time.sleep(1)

            # extract abstract
            abstract = driver.find_element_by_xpath("//div[@class='name-section']/p").text
            info['abstract'] = abstract

            time.sleep(1)

            # extract date
            date = driver.find_elements_by_xpath("//div[@class='name-section']/a[@data-appinsights-action='OpenVenueDetails']/span[@class='year']")[0].text
            info['date'] = date

            time.sleep(1)

            # extract authors
            author_list = driver.find_elements_by_xpath("//div[@class='authors']")[0].text
            author_list = ''.join(i for i in author_list if not i.isdigit())
            author_list = author_list.strip()
            author_list = author_list.split(" , ")
            info['authors'] = author_list

            time.sleep(1)

            # extract related_topic
            topic = driver.find_elements_by_xpath("//div[@class='text au-target']")
            topics = []
            for k in range(3):
                try:
                    topics.append(topic[k].text)
                except:
                    pass
            info['related topic'] = topics

            time.sleep(1)

            # extract citation count and reference count
            re_ci = driver.find_elements_by_xpath("//div[@class='ma-statistics-item au-target']/div[@class='data']/div[@class='count']")
            reference_count = re_ci[0].text
            citation_count = re_ci[1].text
            info['reference_count'] = reference_count
            info['citation_count'] = citation_count

            time.sleep(1)

            ##### driver.find_elements_by_xpath("//div[@class='routes-group']/ma-call-to-action")[1].click()


            # extract references
            results_section = driver.find_element_by_xpath("//div[@class='results']")
            papers = results_section.find_elements_by_xpath("//a[@class='title au-target']")
            links = []
            try:
                links = [paper.get_attribute('href') for paper in papers]
            except:
                pass
            if len(links) >= 10:
                links = links[:10]
            referenced_ids = []
            if reference_count != 0:
                for k in range(len(links)):
                    link = links[k]
                    link_id = re.search(r'^(https://)?(academic\.microsoft\.com/)?(paper/)?(\d+)', link).group(4)
                    if link_id in ids:
                        continue
                    link = "https://academic.microsoft.com/paper/" + str(link_id)
                    referenced_ids.append(link_id)
                    with open("start.txt", "a") as file2:
                        file2.write("\n")
                        file2.write(link)
            info['references'] = referenced_ids

            time.sleep(4)

            List.append(info)

            i += 1
            json_length +=1

            if json_length % 50 == 0:
                print(json_length, 'papers crawled')

        except Exception as e:
            print(e)
            print('****************************************************')
            continue

    with open('CrawledPapers.json', 'w') as file:
        json_string = json.dumps(List, default=lambda o: o.__dict__, indent=2)
        file.write(json_string)

    print("crwaling finished!")

# 2

In [5]:
class Graph:
    def __init__(self):
        self.nodes = []
    
    def add_new_edge(self, parent_id, child_ids):
        parent = self.find_node(parent_id)
        for child_id in child_ids:
            child = self.find_node(child_id)
            parent.add_child(child)
            child.add_parent(parent)
        
    def find_node(self, id):
        response, node = self.is_node_in_graph(id)
        if response:
             return node
        else:
            new_node = Node(id)
            self.nodes.append(new_node)
            return new_node
    
    def is_node_in_graph(self, id):
        for node in self.nodes:
            if id == node.id:
                return True, node
        return False, None
    

class Node:
    def __init__(self, id):
        self.id = id
        self.page_rank = 1
        self.children = []
        self.parents = []
       
    def len_children(self):
        return len(self.children)
    
    def len_parents(self):
        return len(self.parents)
    
    def update_page_rank(self, d, n):
        up_neighbors = self.parents
        sum_ = 0
        for node in up_neighbors:
            try:
                sum_ += node.page_rank/self.len_children()
            except:
                pass
        self.page_rank = (d/n) + (1-d)*sum_
    
    def add_child(self, new_child):
        for child in self.children:
            if child.id == new_child.id:
                return
        self.children.append(new_child)
        
    def add_parent(self, new_parent):
        for parent in self.parents:
            if parent.id == new_parent.id:
                return
        self.parents.append(new_parent)
     

In [6]:
def create_page_ranks(graph, d=0.15, iteration=100):
    for i in range(iteration):
        nodes = graph.nodes
        n = len(nodes)
        for node in nodes:
            node.update_page_rank(d, n)

    page_ranks = dict()
    for node in graph.nodes:
        page_ranks[node.id] = node.page_rank

    with open('PageRank.json', 'w') as fp:
            json.dump(page_ranks, fp)

In [7]:
def pagerank():
    graph = Graph()

    with open(crawled_path) as json_file:
        json_data = json.load(json_file)
        for obj in json_data:
            parent_id = obj['id']
            child_ids = obj['references']
            graph.add_new_edge(parent_id, child_ids)

    create_page_ranks(graph)

# 3

In [8]:
class Authors_Graph:
    def __init__(self):
        self.nodes = []
    
    def add_new_edge(self, parent_name, child_names):
        parent = self.find_node(parent_name)
        for child_name in child_names:
            child = self.find_node(child_name)
            parent.add_child(child)
            child.add_parent(parent)
        
    def find_node(self, name):
        response, node = self.is_node_in_graph(name)
        if response:
             return node
        else:
            new_node = Author_Node(name)
            self.nodes.append(new_node)
            return new_node
    
    def is_node_in_graph(self, name):
        for node in self.nodes:
            if name == node.name:
                return True, node
        return False, None
    
    def normalize_hub_auth(self):
        sum_hub = sum(node.hub for node in self.nodes)
        sum_auth = sum(node.auth for node in self.nodes)
        for node in self.nodes:
            node.hub = node.hub/sum_hub
            node.auth = node.auth/sum_auth
        

class Author_Node:
    def __init__(self, name):
        self.name = name
        self.hub = 1
        self.auth = 1
        self.children = []
        self.parents = []
       
    def len_children(self):
        return len(self.children)
    
    def len_parents(self):
        return len(self.parents)
    
    def add_child(self, new_child):
        for child in self.children:
            if child.name == new_child.name:
                return
        self.children.append(new_child)
        
    def add_parent(self, new_parent):
        for parent in self.parents:
            if parent.name == new_parent.name:
                return
        self.parents.append(new_parent)
    
    def update_hub(self):
        res = 0
        for node in self.children:
            res += node.auth
        self.hub = res
    
    def update_auth(self):
        res = 0
        for node in self.parents:
            res += node.hub
        self.auth = res    

In [9]:
def get_n_best_authors(authors_graph, n_best=10, iteration=5):
    for i in range(iteration):
        nodes = authors_graph.nodes
        n = len(nodes)
        for node in nodes:
            node.update_auth()
        for node in nodes:
            node.update_hub()
        authors_graph.normalize_hub_auth()
    authors_authority = dict()
    for node in authors_graph.nodes:
        authors_authority[node.name] = node.auth
    authors_authority = {k: v for k, v in sorted(authors_authority.items(), key=lambda item: item[1])}
    return list(authors_authority.items())[-n_best:]

In [10]:
def HITS_algorithm():
    paper_authors = dict()
    paper_references = dict()
            
    with open(crawled_path) as json_file:
        json_data = json.load(json_file)
        for obj in json_data:
            id = obj['id']
            authors = obj['authors']
            references = obj['references']
            paper_authors[id] = authors
            paper_references[id] = references

    authors_graph = Authors_Graph()
    for id, authors in paper_authors.items():
        references = paper_references[id]
        for parent_author in authors:
            for reference_id in references:
                try:
                    authors_graph.add_new_edge(parent_author, paper_authors[reference_id])
                except:
                    pass
    return get_n_best_authors(authors_graph)

# 4

In [11]:
df = pd.read_csv('data.csv')

## 4.1

In [12]:
all_related_topics = set()
all_related_topics_count = dict()

class Paper:
    def __init__(self, params):
        self.id = params['id']
        self.related_topics = params['related topic']    ###
        
        global all_related_topics
        for related_topic in self.related_topics:
            all_related_topics.add(related_topic)
            try:
                all_related_topics_count[related_topic] += 1
            except:
                all_related_topics_count[related_topic] = 1
    
    def set_vector(self, vector):
        self.vector = vector

In [13]:
papers = []     
json_data = []
with open(crawled_path) as json_file:
    json_data = json.load(json_file)
    for obj in json_data:
        paper = Paper(obj)
        papers.append(paper)

In [14]:
all_related_topics_count = list(all_related_topics_count)

In [15]:
all_related_topics = list(all_related_topics)
all_related_topics = [x.lower() for x in all_related_topics]
df_columns = df.columns
df_columns = [x.lower() for x in df_columns]
df_columns = [x.strip() for x in df_columns]

swapping_dict = {
    'derivative-free optimization':'algorithms',
    'ibm pc compatible':'computer hardware',
    'structural classification of proteins database':'ai',
    'margin (machine learning)':'ai',
    'iterative deepening depth-first search':'algorithms',
    'linear hashing':'algorithms',
    'recurrent neural network language models':'ai',
    'hill climbing':'algorithms',
    'computer science':'theoretical computer science',
    'artificial intelligence':'ai',
    'mathematics':'theoretical computer science',
    'biology':'theoretical computer science',
    'pattern recognition':'ai',
    'machine learning':'ai',
    'artificial neural network':'ai',
    'algorithm':'algorithms',
    'computer vision':'ai',
    'speech recognition': 'natural language processing',
    'mathematical optimization':'algorithms',
    'information retrieval':'natural language processing',
    'language model':'natural language processing',
    'cognitive neuroscience of visual object recognition':'theoretical computer science',
    'probabilistic logic':'theoretical computer science',
    'unsupervised learning':'ai',
    'recurrent neural network':'ai',
    'word (computer architecture)':'computer hardware',
    'statistics':'theoretical computer science',
    'backpropagation':'ai',
    'machine translation':'ai',
    'set (abstract data type)':'theoretical computer science',
    'deep learning':'ai',
    'feature extraction':'ai',
    'semi-supervised learning':'ai',
    'support vector machine':'ai',
    'applied mathematics':'theoretical computer science',
    'object detection':'ai',
    'cluster analysis':'theoretical computer science',
    'image processing':'ai',
    'inference':'algorithms',
    'pattern recognition (psychology)':'ai',
    'parsing':'natural language processing',
    'task (project management)':'software engineering',
    'context (language use)':'software engineering',
    'linguistics':'natural language processing',
    'sentence':'natural language processing',
    'discrete mathematics':'theoretical computer science',
    'segmentation':'natural language processing',
    'representation (mathematics)':'theoretical computer science',
    'natural language':'natural language processing',
    'structure (mathematical logic)':'software engineering',
    'training set':'ai',
}


In [17]:
for paper in papers:
    main_topics = []
    vector = {x:0 for x in df_columns}
    for topic in paper.related_topics:
        topic = topic.lower()
        if topic in df_columns:
            vector[topic] = 1
        else:
            try:
                key = topic
                value = swapping_dict[key]
                vector[value] = 1
            except:
                pass
        paper.set_vector(vector)

In [18]:
def content_based_recommender(user_profile, papers=papers):                 # user_profile is pandas series
    user_profile = user_profile.fillna(0)
    user_profile = dict(user_profile)
    papers_scores = dict()
    for paper in papers:
        score = np.dot(np.array(list(paper.vector.values())), np.array(list(user_profile.values())))
        papers_scores[paper.id] = score
    papers_scores = {k: v for k, v in sorted(papers_scores.items(), key=lambda item: item[1], reverse=True)}
    return list(papers_scores.items())[:10]

In [19]:
content_based_recommender(df.loc[8], papers)

[('2952632681', 0.6800441014466138),
 ('78159342', 0.6800441014466138),
 ('2964153729', 0.6094595951857205),
 ('2145094598', 0.6094595951857205),
 ('2147800946', 0.6094595951857205),
 ('2899771611', 0.44820417038369104),
 ('2129116707', 0.44820417038369104),
 ('2795241978', 0.44820417038369104),
 ('1997011019', 0.44820417038369104),
 ('3012395598', 0.44820417038369104)]

## 4.2

In [20]:
df_copy = df.copy()

In [21]:
df_copy = df.copy()
df_copy = df_copy.fillna(0)

def collaborative_filtering_recommender(user_profile_id, N):
    global df_copy, papers
    user_profile = df_copy.loc[user_profile_id]
    other_user_profiles_sim = dict()
    
    for i in range(len(df_copy)):
        if user_profile_id != i:
            other_user_profiles_sim[i] = np.dot(np.array(df_copy.loc[i]), user_profile)
    
    other_user_profiles_sim = {k: v for k, v in sorted(other_user_profiles_sim.items(), key=lambda item: item[1], reverse=True)}
    selected_users_id = list(other_user_profiles_sim.keys())[:N]
        
    new_df = df_copy.loc[selected_users_id]
    new_df_mean = new_df.mean()
    
    columns = user_profile.index
    new_user_profile = pd.Series(index=columns, data=np.zeros(len(columns)))
    for c in columns:
        if user_profile[c] == 0.000000:
            new_user_profile[c] = new_df_mean[c]
        else:
            new_user_profile[c] = user_profile[c]
    new_user_profile = new_user_profile / np.sum(new_user_profile)
    recomended_papers = content_based_recommender(new_user_profile, papers)
    return(new_user_profile, recomended_papers)
    

# 4.3

In [22]:
A = df.fillna(0).to_numpy()
x = A.nonzero()[0]
y = A.nonzero()[1]

length = len(x)
indices = np.arange(length)
np.random.shuffle(indices)
train_indices = indices[:int(0.8*length)]
test_indices = indices[int(0.8*length):]

x_train_indices = x[train_indices]
y_train_indices = y[train_indices] 

x_test_indices = x[test_indices]
y_test_indices = y[test_indices]

A_train = np.zeros(A.shape)
A_test = np.zeros(A.shape)

for i in range(len(x_train_indices)):
    x = x_train_indices[i]
    y = y_train_indices[i]
    A_train[x, y] = A[x, y]

for i in range(len(x_test_indices)):
    x = x_test_indices[i]
    y = y_test_indices[i]
    A_test[x, y] = A[x, y]
    

## Calculating Cost

In [23]:
def calculate_cost(A, P, Q, k):
    n = A.shape[0]
    m = A.shape[1]
    return np.sum(np.power(A - np.dot(P, Q), k))

## Gradient Descent

In [24]:
def gradient_descent(A, P, Q, k, lr=0.0001, s=5000):
    n, m = A.shape
    
    # updating P
    for t in range(k):
        for i in range(n):
            P[i, t] -= 2 * lr * np.sum(np.matmul((A[i, :] - np.dot(P[i, :], Q)), Q[t, :]))

    # updating Q
    for t in range(k):
        for j in range(m):
            Q[t, j] -= 2 * lr * np.sum(np.matmul((A[:, j] - np.dot(P, Q[:, j])), P[:, t]))
            
    return P, Q

In [25]:
U, S, V = svd(A_train, full_matrices=False)
U = U[:, :2]
S = S[:2]
V = V[:2, :]

P, Q = gradient_descent(A_train, U.copy(), V.copy(), U.shape[1])

In [26]:
print(calculate_cost(A_train, U, V, 2))
print(calculate_cost(A_train, P, Q, 2))

3066.953190770168
3067.650456421008


In [27]:
print(calculate_cost(A_test, U, V, 2))
print(calculate_cost(A_test, P, Q, 2))

803.9800222672209
803.974835751723


# 5

In [28]:
sg.theme('dark grey 9')

layout = [[sg.Button('crawl!'), sg.Button('calculate pagerank'), sg.Button('calculate HITS')],
         [sg.Text("enter user id:")],
         [sg.InputText(size=(12,)), sg.Button('content based recommender')],
         [],
         [sg.Text("enter user id:"), sg.Text("enter N:")] ,
         [sg.InputText(size=(12,)), sg.InputText(size=(12,)), sg.Button('collabrative filtering recommender')]]

event, values = sg.Window('List Comprehensions', layout).read(close=True)

if event == 'crawl!':
    crawl()
    sg.popup("Crawling Finished! and Results saved in", crawled_path)

    
if event == 'calculate pagerank':
    pagerank()
    sg.popup("PageRanking Algorithm finished and results saved in PageRank.json")

    
if event == 'calculate HITS':
    best_authors = HITS_algorithm()
    authors = [x[0] for x in best_authors][::-1]
    scores = [x[1] for x in best_authors][::-1]
    layout = []
    
    sz=(10,20)
    col1=[[sg.Frame('Authors',[[sg.Text(x)] for x in authors], background_color='#68768C', size=sz)]]
    col2=[[sg.Frame('Authority',[[sg.Text(round(x, 4))] for x in scores], background_color='#2E3E58', size=sz)]]

    layout = [[sg.Column(col1, element_justification='c' ), sg.Column(col2, element_justification='c')]]

    window =sg.Window("HITS",layout)
    event,values=window.read()

    window.close()   
    
    
if event == 'content based recommender':
    id = int(values[0])
    recommends = content_based_recommender(df.loc[id])
    
    sz=(10,20)
    col1=[[sg.Frame('Paper id',[[sg.Text(x[0])] for x in recommends], background_color='#68768C', size=sz)]]
    col2=[[sg.Frame('Score',[[sg.Text(round(x[1], 4))] for x in recommends], background_color='#2E3E58', size=sz)]]

    layout = [[sg.Column(col1, element_justification='c' ), sg.Column(col2, element_justification='c')]]

    window =sg.Window('Recomended Papers',layout)
    event,values=window.read()
    print(event)

    window.close() 
    
    
if event == 'collabrative filtering recommender':
    id = int(values[1])
    N = int(values[2])
    new_user_profile, recommends = collaborative_filtering_recommender(id, N)
    
    new_user_profile = {k:v for k,v in sorted(new_user_profile.items(), key=lambda item: item[1], reverse=True)}

    sz1=(10,30)
    col1=[[sg.Frame('Paper id',[[sg.Text(x)] for x in list(new_user_profile.keys())[:20]], background_color='#68768C', size=sz1)]]
    col2=[[sg.Frame('Score',[[sg.Text(round(x, 4))] for x in list(new_user_profile.values())[:20]], background_color='#2E3E58', size=sz1)]]

    layout = [[sg.Column(col1, element_justification='c' ), sg.Column(col2, element_justification='c')],
             [sg.Button("Recommends")]]

    window =sg.Window('user new profile and recomended papers',layout)
    event,values=window.read(close=True)
    
    sz=(10,20)
    col1=[[sg.Frame('Paper id',[[sg.Text(x[0])] for x in recommends], background_color='#68768C', size=sz)]]
    col2=[[sg.Frame('Score',[[sg.Text(round(x[1], 4))] for x in recommends], background_color='#2E3E58', size=sz)]]

    layout = [[sg.Column(col1, element_justification='c' ), sg.Column(col2, element_justification='c')]]

    window =sg.Window('Recomended Papers',layout)
    event,values=window.read()

    window.close()
